In [ ]:
import pickle
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import cv2
import matplotlib.pyplot as plt
import tensorflow
from annoy import AnnoyIndex

Load feature embeddings and filenames

In [ ]:
feature_list = np.array(pickle.load(open('embeddings_colab.pkl', 'rb')))
filenames = pickle.load(open('filenames_colab.pkl', 'rb'))

Load and preprocess query image

In [ ]:
query_img = image.load_img('sample/GenAI1.png', target_size=(224, 224))
query_img_array = image.img_to_array(query_img)
expanded_query_img_array = np.expand_dims(query_img_array, axis=0)
preprocessed_query_img = preprocess_input(expanded_query_img_array)

Load or create the ResNet50 model

In [ ]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

In [ ]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

Extract features from the query image

In [ ]:
query_features = model.predict(preprocessed_query_img).flatten()
normalized_query_features = query_features / norm(query_features)

Build Annoy index

In [ ]:
annoy_index = AnnoyIndex(normalized_query_features.shape[0], metric='euclidean')
for idx, feature_vector in enumerate(feature_list):
    annoy_index.add_item(idx, feature_vector)
annoy_index.build(10)  # Number of trees in the index

Perform nearest neighbors search

In [ ]:
num_neighbors = 6
nearest_indices = annoy_index.get_nns_by_vector(normalized_query_features, num_neighbors)

Visualize similar images

In [ ]:
plt.figure(figsize=(15, 8))

In [ ]:
for idx, file_idx in enumerate(nearest_indices[1:], start=1):
    similar_img = cv2.imread(filenames[file_idx])
    print(filenames[file_idx])
    similar_img = cv2.cvtColor(similar_img, cv2.COLOR_BGR2RGB)
    plt.subplot(1, num_neighbors - 1, idx)
    plt.imshow(similar_img)
    plt.title(f"Similar {idx}")
    plt.axis('off')

In [ ]:
plt.tight_layout()
plt.show()